In [1]:
from datetime import datetime as dt
from collections import defaultdict
import time
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter
import statsmodels.api as sm
from sympy import *
import pytz
import itertools
import matplotlib.pyplot as plt
from itertools import combinations
from itertools import product
from itertools import groupby
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [2]:
if not mt5.initialize(path="C:/MetaTrader_A/MetaTrader 5/terminal64.exe",login=1049732, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [3]:
def MetaLink(p,x,y,z):
    if x == "D1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_D1, z, y)
    elif x == "H8":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H8, z, y)
    elif x == "H4":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H4, z, y)
    elif x == "H3":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H3, z, y)
    elif x == "H2":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H2, z, y)
    elif x == "H1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_H1, z, y)
    elif x == "M30":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M30, z, y)
    elif x == "M15":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M15, z, y)
    elif x == "M5":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M5, z, y)
    elif x == "M1":
        data = mt5.copy_rates_from(p, mt5.TIMEFRAME_M1, z, y)
    return data

In [4]:
def MetaLink_range(p,x,y,z):
    if x == "D1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_D1, y, z)
    elif x == "H8":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H8, y, z)
    elif x == "H4":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H4, y, z)
    elif x == "H3":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H3, y, z)
    elif x == "H2":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H2, y, z)
    elif x == "H1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_H1, y, z)
    elif x == "M30":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M30, y, z)
    elif x == "M15":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M15, y, z)
    elif x == "M5":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M5, y, z)
    elif x == "M1":
        data = mt5.copy_rates_range(p, mt5.TIMEFRAME_M1, y, z)
    return data

In [5]:
def MetaLink_pos(p,x,y):
    if x == "D1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_D1, 0, y)
    elif x == "H8":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H8, 0, y)
    elif x == "H4":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H4, 0, y)
    elif x == "H3":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H3, 0, y)
    elif x == "H2":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H2, 0, y)
    elif x == "H1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_H1, 0, y)
    elif x == "M30":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M30, 0, y)
    elif x == "M15":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M15, 0, y)
    elif x == "M5":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M5, 0, y)
    elif x == "M1":
        data = mt5.copy_rates_from_pos(p, mt5.TIMEFRAME_M1, 0, y)
    return data

In [6]:
def Pip_Screen(a,d):
    
    base, quote = a[:3], a[3:]

    if quote == "USD":
        exchange = 1.00
        if base == "XAG":
            D = d*10000
        elif base == "XAU":
            D = d*100
        elif base == "BTC":
            D = d*10
        elif base == "BCH":
            D = d*100
        elif base == "ETH":
            D = d*10
        elif base == "DSH":
            D = d*100
        elif base == "LTC":
            D = d*100
        elif base == "XMR":
            D = d*100
        elif base == "ZEC":
            D = d*100
        elif base == "ETC":
            D = d*1000
        elif base == "XRP":
            D = d*10000
        else:
            D = d*10000
    if quote == "USDt":
        exchange = 1.00
        if base == "BTC":
            D = d*10
    elif quote == "AUD":
        exchange = 1/(mt5.symbol_info("AUDUSD").bid)
        D = d*10000
    elif quote == "CAD":
        exchange = mt5.symbol_info("USDCAD").bid
        D = d*10000
    elif quote == "CHF":
        exchange = mt5.symbol_info("USDCHF").bid
        D = d*10000
    elif quote == "EUR":
        exchange = 1/(mt5.symbol_info("EURUSD").bid)
        D = d*10000
    elif quote == "GBP":
        exchange = 1/(mt5.symbol_info("GBPUSD").bid)
        D = d*10000
    elif quote == "JPY":
        exchange = mt5.symbol_info("USDJPY").bid
        D = d*100
    elif quote == "NZD":
        exchange = 1/(mt5.symbol_info("NZDUSD").bid)
        D = d*10000
    elif quote == 'NOK':
        exchange = mt5.symbol_info("USDNOK").bid
        D = d*1000
    elif quote == 'SEK':
        exchange = mt5.symbol_info("USDSEK").bid
        D = d*1000
    elif quote == 'SGD':
        exchange = mt5.symbol_info("USDSGD").bid
        D = d*10000
    elif quote == 'DKK':
        exchange = mt5.symbol_info("USDDKK").bid
        D = d*1000
    elif quote == 'MXN':
        exchange = mt5.symbol_info("USDMXN").bid
        D = d*1000
    elif quote == "entCrude":
        exchange = 1.00
        D = d*10
    elif quote == "ICrude":
        exchange = 1.00
        D = d*10
    elif quote == "TRY":
        exchange = mt5.symbol_info("USDTRY").bid
        D = d*10000
        
    return D
    

In [7]:
def Pip_ReverseScreen(a,d):
    
    base, quote = a[:3], a[3:]

    if quote == "USD":
        if base == "XAG":
            D = d/10000
        elif base == "XAU":
            D = d/100
        elif base == "BTC":
            D = d/10
        elif base == "BCH":
            D = d/100
        elif base == "ETH":
            D = d/10
        elif base == "DSH":
            D = d/100
        elif base == "LTC":
            D = d/100
        elif base == "XMR":
            D = d/100
        elif base == "ZEC":
            D = d/100
        elif base == "ETC":
            D = d/1000
        elif base == "XRP":
            D = d/10000
        else:
            D = d/10000
    if quote == "USDt":
        if base == "BTC":
            D = d/10
    elif quote == "AUD":
        D = d/10000
    elif quote == "CAD":
        D = d/10000
    elif quote == "CHF":
        D = d/10000
    elif quote == "EUR": 
        D = d/10000
    elif quote == "GBP":
        D = d/10000
    elif quote == "JPY":
        D = d/100
    elif quote == "NZD":
        D = d/10000
    elif quote == 'NOK':
        D = d/1000
    elif quote == 'SEK':
        D = d/1000
    elif quote == 'SGD':
        D = d/10000
    elif quote == 'DKK':
        D = d/1000
    elif quote == 'MXN':
        D = d/1000
    elif quote == "entCrude":
        exchange = 1.00
        D = d/10
    elif quote == "ICrude":
        exchange = 1.00
        D = d/10
    elif quote == "TRY":
        D = d/10000
        
    return D

In [8]:
def positioner(a,b,c,d,e):
    
    riskusd = 50.00        

    base, quote = a[:3], a[3:]
    
    ro = 2

    if quote == "USD":
        exchange = 1.00
        if base == "XAG":
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/1/stpips,ro)
        elif base == "XAU":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "BTC":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "BCH":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ETH":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "DSH":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "LTC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "XMR":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ZEC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ETC":
            stpips = abs(e - c)*1000
            lot = round(((riskusd*exchange)/10)/10/stpips,ro)
        elif base == "XRP":
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/1/stpips,ro)
        else:
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/stpips,ro)
    if quote == "USDt":
        exchange = 1.00
        if base == "BTC":
            stpips = abs(e - c)*10
            lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "AUD":
        exchange = 1/(mt5.symbol_info("AUDUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "CAD":
        exchange = mt5.symbol_info("USDCAD").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "CHF":
        exchange = mt5.symbol_info("USDCHF").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "EUR":
        exchange = 1/(mt5.symbol_info("EURUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "GBP":
        exchange = 1/(mt5.symbol_info("GBPUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "JPY":
        exchange = mt5.symbol_info("USDJPY").bid
        stpips = abs(e - c)*100
        lot = round(((riskusd*exchange)/10)/100/stpips,ro)
    elif quote == "NZD":
        exchange = 1/(mt5.symbol_info("NZDUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == 'NOK':
        exchange = mt5.symbol_info("USDNOK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'SEK':
        exchange = mt5.symbol_info("USDSEK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'SGD':
        exchange = mt5.symbol_info("USDSGD").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == 'DKK':
        exchange = mt5.symbol_info("USDDKK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'MXN':
        exchange = mt5.symbol_info("USDMXN").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == "entCrude":
        exchange = 1.00
        stpips = abs(e - c)*10
        lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "ICrude":
        exchange = 1.00
        stpips = abs(e - c)*10
        lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "TRY":
        exchange = mt5.symbol_info("USDTRY").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
        
    deviation = 20
    
    if b == "LONG":
        price = mt5.symbol_info_tick(a).ask
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "LONDON",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
    elif b == "SHORT":
        price = mt5.symbol_info_tick(a).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "LONDON",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"
            
    lotflag = "GREEN"
    if lot == 0:
        lotflag = "RED"
    
    return request, lotflag

In [9]:
def LONDON(pair):
    
    for p in pair:
    
        D = MetaLink_pos(p,"M5",100)
        STy = Pip_Screen(p,D['close'])
        STx = range(0,len(STy))
        slope, intercept, r_value, p_value, std_err = stats.linregress(STx,STy)
        ST_slope = slope
        scopetheta = abs(math.degrees(math.atan(ST_slope)))
        if scopetheta > 30:

            S = savgol_filter(STy, 21, 3)

            tops = S[argrelextrema(S, np.greater)[0]]
            slope, intercept, r_value, p_value, std_err = stats.linregress(range(0,len(tops)),tops)
            tops_slope = slope

            bottoms = S[argrelextrema(S, np.less)[0]]
            slope, intercept, r_value, p_value, std_err = stats.linregress(range(0,len(bottoms)),bottoms)
            bottoms_slope = slope

            if tops_slope > bottoms_slope:
                denom = bottoms_slope
            else:
                denom = tops_slope
            slope_variance = abs((tops_slope - bottoms_slope)/denom)
            if slope_variance < 0.10:

                if ST_slope > 0:
                    if D['close'][-1] < bottoms[-1]:
                        position = "LONG"
                        entry = D['close'][-1]
                        takeprofit = tops[-1]
                        tpdist = takeprofit - entry
                        stoploss = entry - tpdist/2
                    else:
                        position = 0
                else:
                    if D['close'][-1] > tops[-1]:
                        position = "SHORT"
                        entry = D['close'][-1]
                        takeprofit = bottoms[-1]
                        tpdist = entry - takeprofit
                        stoploss = entry + tpdist/2
                    else:
                        position = 0

                if position != 0:
                    request, lotflag = positioner(p,position,entry,takeprofit,stoploss)
                    if lotflag == "GREEN":
                        result = mt5.order_send(request)
                        if result.retcode != mt5.TRADE_RETCODE_DONE:
                            print("order_send failed, retcode={}".format(result.retcode))
                            result_dict=result._asdict()
                            for field in result_dict.keys():
                                print("   {}={}".format(field,result_dict[field]))
                                if field=="request":
                                    traderequest_dict=result_dict[field]._asdict()
                                    for tradereq_filed in traderequest_dict:
                                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
                        else:
                            print("Trade Placed:   ",p,"   ","Long","   TP:  ",takeprofit,"   SL:  ",stoploss)
                            

In [10]:
print("Initializing...")
pair = []
symbols=mt5.symbols_get()
for s in symbols:
    pair.append(s.name)
    
t2 = 0
while True:
    Drates = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_M5, 0, 1)
    D = Drates[0]
    t = D[0]
    if t2 != 0:
        if t != t2:
            print("Analyzing Markets...")
            LONDON(pair)
    t2 = t
    time.sleep(1)

Initializing...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...
Analyzing Markets...


KeyboardInterrupt: 